In [47]:
import numpy as np
import math as m

In [48]:

import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import json
import os

### Import Transformed Markers

In [49]:
markers_df = pd.read_csv('model_update/tps_warping_results/markers_transformed.csv', index_col=0)

In [50]:
markers_df[markers_df['body']=='tibia_r']

,body,location
name,,
ankle_r_lat,tibia_r,[-0.00985172 -0.38197933 0.0274688 ]
ankle_r_med,tibia_r,[ 0.00982868 -0.37200758 -0.02746111]
knee_r_center,tibia_r,[0. 0. 0.]
tibia_r_center,tibia_r,[-2.18326114e-05 -3.86553332e-02 3.07296020e-03]
ankle_r_center,tibia_r,[-1.15226842e-05 -3.76993453e-01 3.84092081e-06]
fibula_r_as,tibia_r,[ 0.0013508 -0.05575801 0.03088342]
fibula_r_sup,tibia_r,[-0.01883829 -0.04092043 0.04093777]
tibia_r_midshaft_anter,tibia_r,[ 0.02554335 -0.18849747 0.00347409]
tibia_r_midshaft_poster,tibia_r,[-0.00265919 -0.18849673 -0.00807731]


### Import Transformed Muscle Paths

In [51]:
mscles_df = pd.read_csv('model_update/tps_warping_results/muscles_transformed.csv', index_col=0)
mscles_df.head()

,body,location
name,,
addbrev_r-P1,pelvis,[-0.02278461 -0.08613435 0.0164788 ]
addlong_r-P1,pelvis,[-0.01314173 -0.07989293 0.01990555]
addmagDist_r-P1,pelvis,[-0.05272527 -0.12135554 0.03481227]
addmagIsch_r-P1,pelvis,[-0.06603244 -0.12417136 0.0370318 ]
addmagMid_r-P1,pelvis,[-0.04014287 -0.11391036 0.02635751]


### Import Transformed Wrapping Surfaces

In [52]:
wrp_df = pd.read_csv('model_update/tps_warping_results/wrp_transformed.csv', index_col=1)
wrp_df.head()

,body,translation,rotation,length,radius
name,,,,,
Gmax1_at_pelvis_r,pelvis,"[-0.064, -0.0947, 0.0652]","[-0.55454, -0.10539, 0.00092]",0.1662,0.0329
Gmax2_at_pelvis_r,pelvis,"[-0.0692, -0.0799, 0.0748]","[-0.71816, -0.10412, 0.00064]",0.1040,0.0340
Gmax3_at_pelvis_r,pelvis,"[-0.0714, -0.0848, 0.0744]","[-0.16217, -0.04009, 0.00019]",0.1074,0.0343
Gmax1_at_pelvis_l,pelvis,"[-0.0608, -0.0917, -0.0666]","[0.54819, 0.13154, 0.00179]",0.1808,0.0328
Gmax2_at_pelvis_l,pelvis,"[-0.0666, -0.0765, -0.0754]","[0.67664, 0.13707, 0.00157]",0.1150,0.0344


### Parse current scaled model

In [53]:
tree=ET.parse("scaled_model_P03.osim")
root = tree.getroot()

### Update reference geometry

In [54]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\r_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\l_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\sacrum.stl')

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_r.stl')
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_l.stl')

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_r.stl')
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_r.stl')

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_l.stl')
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_l.stl')

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_r.stl')
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_l.stl')

In [55]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [56]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [57]:
for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']
    if point in mscles_df.index:
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        # print(point, new_text)
        PathPoint.find('location').text = new_text
    else: pass

### Update locations of muscle wrapping surfaces

In [58]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'translation']
        new_text = translation[1:-1]
        # print(point, new_text)
        WrapCylinder.find('translation').text = new_text
    else: pass

### Update Translation of joint centres

In [59]:
torso_origin_in_pelvis = markers_df.loc['torso_origin_in_pelvis', 'location']

femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [60]:
for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    #ground_pelvis
    if joint.attrib['name'] ==  "back":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = torso_origin_in_pelvis[1:-1]
                print('old torso origin translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new torso origin translation', frame.find('translation').text)
            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_r_center_in_pelvis[1:-1]
                print('old r hip translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r hip translation', frame.find('translation').text)
            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                print('old r walker knee translation in femur',frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r walker knee translation in femur',frame.find('translation').text)
            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    print ('old r tibia translation', frame.find('translation').text)
            #    frame.find('translation').text = new_text
            #    print ('new r tibia translation', frame.find('translation').text)
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                frame.find('translation').text = new_text
    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":
               new_text = knee_r_center_in_femur[1:-1]
               print('patellofemoral r old', frame.find('translation').text)
               frame.find('translation').text = new_text
               print('patellofemoral r new', frame.find('translation').text)
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":
               new_text = knee_l_center_in_femur[1:-1]
               frame.find('translation').text = new_text
           else: pass

for joint in root.iter('PinJoint'):
    
    name = joint.attrib['name']

    if joint.attrib['name'] ==  "ankle_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_r_offset":
               new_text = talus_r_center_in_tibia [1:-1]
               print('ankle r old', frame.find('translation').text)
               frame.find('translation').text = new_text
               print('ankle r new', frame.find('translation').text)
           else: pass
    
    elif joint.attrib['name'] ==  "ankle_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_l_offset":
               new_text = talus_r_center_in_tibia [1:-1]
               frame.find('translation').text = new_text
           else: pass
    else: pass

old r hip translation -0.051425784270424685 -0.067104381248516018 0.07060125262515124
new r hip translation -0.04935395 -0.07197149  0.08605985
old r walker knee translation in femur -0.007601819806194619 -0.38334220125280055 -0.0025840549403010134
new r walker knee translation in femur  6.44817533e-16 -3.75586939e-01 -2.84217094e-17
patellofemoral r old -0.007601819806194619 -0.38334220125280055 -0.0025840549403010134
patellofemoral r new  6.44817533e-16 -3.75586939e-01 -2.84217094e-17
old torso origin translation -0.09202104762299676 0.069677756042222636 0
new torso origin translation -0.0762313   0.07016325 -0.00247692
ankle r old -0.0097593388562149559 -0.39037355424859826 0
ankle r new  0.00236077 -0.38991173  0.00219256


### Update Markers for Joint Positions in Parent

In [61]:
['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r' , 'ankle_l_tibia_l_offset', 'ankle_r_tibia_r_offset'] #; ankle_l_tibia_l_offset = talus_l_center_in_tibia; ankle_r_tibia_r_offset = talus_r_center_in_tibia

['torso_origin_in_pelvis',
 'femur_l_center_in_pelvis',
 'femur_r_center_in_pelvis',
 'knee_l_center_in_femur_l',
 'patella_l_in_femur_l',
 'knee_r_center_in_femur_r',
 'patella_r_in_femur_r',
 'ankle_l_tibia_l_offset',
 'ankle_r_tibia_r_offset']

In [62]:
m_names = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r']

for marker in root.iter('Marker'):
    name = marker.attrib['name']
    print(name)
    if name == 'ankle_l_tibia_l_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['ankle_l_center', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name == 'ankle_r_tibia_r_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['ankle_r_center', 'location']
        marker.find('location').text = new_text[1:-1]
    if name in m_names:
        #print(name)
        old_text = marker.find('location').text
        #print(old_text)
        new_text = markers_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]
        #print(marker.find('location').text)

C7
RBAK
CLAV
STRN
T10
RASI
RPSI
LPSI
LASI
PE01
PE02
PE03
RTH1
RTH2
RTH3
RGT
RKNE
RMKNE
LTH1
LTH2
LTH3
LGT
LKNE
LMKNE
RTB1
RTB2
RTB3
RANK
RMMA
LTB1
LTB2
LTB3
LANK
LMMA
RHEE
RD5M
RTOE
LHEE
LD5M
LTOE
torso_origin_in_pelvis
pelvis_origin_in_ground
femur_l_center_in_pelvis
femur_r_center_in_pelvis
knee_l_center_in_femur_l
knee_r_center_in_femur_r
patella_l_in_femur_l
patella_r_in_femur_r
ankle_l_tibia_l_offset
ankle_r_tibia_r_offset


### Export transformed model

In [63]:
tree.write('p03_tps_transformed.osim')